In [2]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize
from standard_plotting import set_size

In [5]:
# indir_track_cascade_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/track_mu_Burnsample_RD_Full_db.csv"
# indir_energy_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/energy_Burnsample_RD_Full_db.csv"
# indir_zenith_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/zenith_Burnsample_RD_Full_db.csv"
# indir_azimuth_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/azimuth_Burnsample_RD_Full_db.csv"
# indir_multiclass_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/pid_Burnsample_RD_Full_db_new_muon.csv"
# indir_position_RD = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/['position_x', 'position_y', 'position_z']_Burnsample_RD_Full_db.csv"
# pulses_strings_doms_RD_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/Pulses_doms_strings_times_burnsample.csv"
# lvl3_variables_RD_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Burnsample/lvl3_variables_burnsample.csv"

#Extra for MC: Retro, Oscweight
indir_track_cascade_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/track_cascade_New_muon_test_set_inc_truth.csv"
indir_energy_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/energy_New_muon_test_set_inc_truth.csv"
indir_zenith_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/zenith_New_muon_test_set.csv"
indir_azimuth_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/azimuth_New_muon_test_set.csv"
indir_multiclass_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/pid_Multiclass_try_2_on_test_new_muon.csv"
indir_position_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/position_vertex_new_muon.csv"
pulses_strings_doms_MC_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/Pulses_doms_strings_times_new_muon_db.csv"
lvl3_variables_MC_indir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/lvl3_variables_new_muon.csv"


In [6]:
track_cascade_MC = pd.read_csv(indir_track_cascade_MC).sort_values('event_no').reset_index(drop = True)
energy_MC = pd.read_csv(indir_energy_MC).sort_values('event_no').reset_index(drop = True)
zenith_MC = pd.read_csv(indir_zenith_MC).sort_values('event_no').reset_index(drop = True)
azimuth_MC = pd.read_csv(indir_azimuth_MC).sort_values('event_no').reset_index(drop = True)
multiclass_MC = pd.read_csv(indir_multiclass_MC).sort_values('event_no').reset_index(drop = True)
position_MC = pd.read_csv(indir_position_MC).sort_values('event_no').reset_index(drop = True)
pulses_strings_doms_MC = pd.read_csv(pulses_strings_doms_MC_indir).sort_values('event_no').reset_index(drop = True)


useful_columns = ['event_no',
       'C2HR6', 'CausalVetoHits', 'CleanedFullTimeLength', 'DCFiducialHits',
       'L3_oscNext_bool', 'NAbove200Hits', 'NchCleaned', 'NoiseEngineNoCharge',
       'RTVetoCutHit', 'STW9000_DTW300Hits', 'UncleanedFullTimeLength',
       'VertexGuessZ', 'VetoFiducialRatioHits']

lvl3_variables_MC = pd.read_csv(lvl3_variables_MC_indir,usecols=useful_columns).sort_values('event_no').reset_index(drop = True)



In [7]:
def to_logit(p):
    eps = 0.0000001
    try:
        if np.isnan(p):
            return
        p = p*(1-2*eps)+eps
        logit = np.log(p/(1-p))
    except ZeroDivisionError as e:
        print(e)
    return logit

multiclass_MC['pid_neutrino_pred_logit'] = pd.Series(multiclass_MC['pid_neutrino_pred']).apply(to_logit)

In [8]:
indir_osc_weight_MC = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/osc_weights_new_muon.csv"
osc_weight_MC = pd.read_csv(indir_osc_weight_MC)
print(len(osc_weight_MC))
print(osc_weight_MC[osc_weight_MC['event_no']==28956376])
osc_weight_MC['osc_weight'][osc_weight_MC['event_no']==28956376] = 0
print(osc_weight_MC[osc_weight_MC['event_no']==28956376])

38885781
          event_no  osc_weight
28956376  28956376         NaN


/tmp/ipykernel_45931/382299087.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osc_weight_MC['osc_weight'][osc_weight_MC['event_no']==28956376] = 0


          event_no  osc_weight
28956376  28956376         0.0


In [9]:
nr_electron_neutrino_i3_files = 602
nr_muon_neutrino_i3_files = 1518
nr_tau_neutrino_i3_files = 334
nr_muon_i3_files = 1093
nr_noise_i3_files = 401

nr_electron_neutrinos_total = 8118575 + 183333
nr_muon_neutrinos_total = 19923619 + 183333
nr_tau_neutrinos_total = 8702748 + 183333
nr_muon_total = 169737 + 550000
nr_noise_total = 321103 + 550000

nr_electron_neutrinos_test = 8118575
nr_muon_neutrinos_test = 19923619
nr_tau_neutrinos_test = 8702748
nr_muon_test = 169737
nr_noise_test =321103

electron_neutrino_add_weight = 1/(nr_electron_neutrino_i3_files*nr_electron_neutrinos_test/nr_electron_neutrinos_total)
muon_neutrino_add_weight = 1/(nr_muon_neutrino_i3_files*nr_muon_neutrinos_test/nr_muon_neutrinos_total)
tau_neutrino_add_weight = 1/(nr_tau_neutrino_i3_files*nr_tau_neutrinos_test/nr_tau_neutrinos_total)
muon_add_weight = 1/(nr_muon_i3_files*nr_muon_test/nr_muon_total)
noise_add_weight = 1/(nr_noise_i3_files*nr_noise_test/nr_noise_total)
print(electron_neutrino_add_weight,muon_neutrino_add_weight,tau_neutrino_add_weight)

electron_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-12,12))].to_list()
muon_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-14,14))].to_list()
tau_neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-16,16))].to_list()
muon_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-13,13))].to_list()
noise_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-1,1))].to_list()
neutrino_event_nos = multiclass_MC['event_no'][multiclass_MC['pid'].isin((-12,12,-14,14,-16,16))].to_list()

print(osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((electron_neutrino_event_nos))].head(10))
osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((electron_neutrino_event_nos))] *= electron_neutrino_add_weight
print(osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((electron_neutrino_event_nos))].head(10))
osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((muon_neutrino_event_nos))] *= muon_neutrino_add_weight
osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((tau_neutrino_event_nos))] *= tau_neutrino_add_weight
osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((muon_event_nos))] *= muon_add_weight
osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((noise_event_nos))] *= noise_add_weight

0.001698641060838674 0.0006648233149566109 0.0030570841456206404
878    1.375828e-07
890    2.086777e-07
904    1.883566e-07
916    2.064462e-07
930    1.458803e-07
932    1.216562e-07
938    3.205147e-07
945    2.292233e-07
946    2.398221e-07
956    1.527254e-07
Name: osc_weight, dtype: float64


/tmp/ipykernel_45931/3908699960.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((electron_neutrino_event_nos))] *= electron_neutrino_add_weight


878    2.337037e-10
890    3.544685e-10
904    3.199503e-10
916    3.506780e-10
930    2.477983e-10
932    2.066502e-10
938    5.444394e-10
945    3.893682e-10
946    4.073716e-10
956    2.594257e-10
Name: osc_weight, dtype: float64


/tmp/ipykernel_45931/3908699960.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((muon_neutrino_event_nos))] *= muon_neutrino_add_weight
/tmp/ipykernel_45931/3908699960.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  osc_weight_MC['osc_weight'][osc_weight_MC['event_no'].isin((tau_neutrino_event_nos))] *= tau_neutrino_add_weight
/tmp/ipykernel_45931/3908699960.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [10]:
print(np.shape(track_cascade_MC))
print(np.shape(energy_MC))
print(np.shape(zenith_MC))
print(np.shape(azimuth_MC))
print(np.shape(multiclass_MC))
print(np.shape(position_MC))
print(np.shape(pulses_strings_doms_MC))
print(np.shape(lvl3_variables_MC))

print(np.shape(osc_weight_MC))



(37235782, 5)
(37235782, 4)
(37235782, 4)
(37235782, 4)
(37235782, 7)
(37235782, 8)
(38885781, 5)
(38885781, 14)
(38885781, 2)


In [16]:
print(track_cascade_MC.head(1))
print(energy_MC.head(1))
print(zenith_MC.head(1))
print(azimuth_MC.head(1))
print(multiclass_MC.head(1))
print(position_MC.head(1))
print(pulses_strings_doms_MC.head(1))
print(lvl3_variables_MC.head(1))

print(osc_weight_MC.head(1))


   track_mu_pred  event_no  track_mu
0       0.448105       0.0         1
   energy_pred  event_no    energy
0     3.793501       0.0  3.170023
   zenith_pred  zenith_kappa  event_no    zenith
0     1.799249      3.846686       0.0  1.586985
   azimuth_pred  azimuth_kappa  event_no   azimuth
0       0.11761       0.618752       0.0  4.110935
   pid_noise_pred  pid_muon_pred  pid_neutrino_pred   pid  event_no  \
0        0.000318       0.000078           0.999604 -14.0       0.0   

   pid_neutrino_pred_logit  
0                 7.834167  
   position_x_pred  position_y_pred  position_z_pred  position_x  position_y  \
0       115.700378       -60.552162      -341.636292  101.276772  -76.372551   

   position_z  event_no  
0 -336.303162       0.0  
   event_no  N_pulses  N_string  N_doms  First_dom_time
0         0        36        21      25    56783.915364
   event_no  C2HR6  CausalVetoHits  CleanedFullTimeLength  DCFiducialHits  \
0         0    1.0             2.0                  6

In [12]:
print(track_cascade_MC.drop(columns=['Unnamed: 0','energy'],inplace=True))
print(energy_MC.drop(columns='Unnamed: 0',inplace=True))
#print(zenith_MC.drop(columns='Unnamed: 0',inplace=True))
#print(azimuth_MC.drop(columns='Unnamed: 0',inplace=True))
print(multiclass_MC.drop(columns='Unnamed: 0',inplace=True))
print(position_MC.drop(columns='Unnamed: 0',inplace=True))
#print(pulses_strings_doms_MC.drop(columns='Unnamed: 0',inplace=True))
#print(lvl3_variables_MC.drop(columns='Unnamed: 0',inplace=True))
#print(retro.drop(columns='Unnamed: 0',inplace=True))
#print(osc_weight_MC.drop(columns='Unnamed: 0',inplace=True))
print(osc_weight_MC.rename(columns={"osc_weight": "total_osc_weight"},inplace=True))

None
None
None
None
None


In [13]:
indir_interaction_type = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Test_set/"
MC_interaction_type = pd.read_csv(indir_interaction_type + 'interaction_type_New_muon_test.csv').sort_values('event_no').reset_index(drop = True)

In [14]:
MC_interaction_type.drop(columns='pid',inplace=True)

In [15]:
print(MC_interaction_type.head(1))
print(np.shape(MC_interaction_type))

   event_no  interaction_type
0         0               1.0
(38885781, 2)


In [17]:
total_MC = pd.merge(track_cascade_MC.reset_index(drop=True),energy_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),zenith_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),azimuth_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),multiclass_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),position_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),pulses_strings_doms_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),lvl3_variables_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),MC_interaction_type.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)
total_MC = pd.merge(total_MC.reset_index(drop=True),osc_weight_MC.reset_index(drop=True),on='event_no',how='inner').reset_index(drop=True)



In [18]:
print(total_MC.columns)

Index(['track_mu_pred', 'event_no', 'track_mu', 'energy_pred', 'energy',
       'zenith_pred', 'zenith_kappa', 'zenith', 'azimuth_pred',
       'azimuth_kappa', 'azimuth', 'pid_noise_pred', 'pid_muon_pred',
       'pid_neutrino_pred', 'pid', 'pid_neutrino_pred_logit',
       'position_x_pred', 'position_y_pred', 'position_z_pred', 'position_x',
       'position_y', 'position_z', 'N_pulses', 'N_string', 'N_doms',
       'First_dom_time', 'C2HR6', 'CausalVetoHits', 'CleanedFullTimeLength',
       'DCFiducialHits', 'L3_oscNext_bool', 'NAbove200Hits', 'NchCleaned',
       'NoiseEngineNoCharge', 'RTVetoCutHit', 'STW9000_DTW300Hits',
       'UncleanedFullTimeLength', 'VertexGuessZ', 'VetoFiducialRatioHits',
       'interaction_type', 'total_osc_weight'],
      dtype='object')


In [20]:
print(total_MC.head(1))
print(np.shape(total_MC))

   track_mu_pred  event_no  track_mu  energy_pred    energy  zenith_pred  \
0       0.448105       0.0         1     3.793501  3.170023     1.799249   

   zenith_kappa    zenith  azimuth_pred  azimuth_kappa  ...  NAbove200Hits  \
0      3.846686  1.586985       0.11761       0.618752  ...            1.0   

   NchCleaned  NoiseEngineNoCharge  RTVetoCutHit  STW9000_DTW300Hits  \
0         5.0                  1.0           1.0                 3.0   

   UncleanedFullTimeLength  VertexGuessZ  VetoFiducialRatioHits  \
0                  10057.0        -242.0                    0.0   

   interaction_type  total_osc_weight  
0               1.0      4.011465e-10  

[1 rows x 41 columns]
(37235782, 41)


In [21]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"

total_MC.to_csv(outdir + 'Monte_Carlo_all_events_all_variables.csv',index=False)

In [ ]:
np.shape(total_MC[total_MC['pid_neutrino_pred_logit']>1])

In [ ]:
small_MC = total_MC[total_MC['pid_neutrino_pred_logit']>1]
small_MC.to_csv(outdir + 'Monte_Carlo_neutrino_prob_logit_above_1_all_variables.csv',index=False)

In [ ]:
outdir = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/"


In [4]:
# test = pd.read_csv("/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/track_cascade_sets/Monte_Carlo_neutrino_prob_logit_above_12_all_variables.csv")
# print(np.shape(test))

(12669623, 41)


In [1]:
small_small_MC = test[test['pid_neutrino_pred_logit']>12]
small_small_MC.to_csv(outdir + 'Monte_Carlo_neutrino_prob_logit_above_12_all_variables.csv',index=False)

NameError: name 'test' is not defined

In [ ]:
print(np.shape(small_small_MC))

(18226, 33)
